# Reference
- https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-1-fd544fab149

# Introduction
Reinforcement learning provides the capacity for us not only to teach an artificial agent how to act, but to allow it to learn through it’s own interactions with an environment. By combining the complex representations that deep neural networks can learn with the goal-driven learning of an RL agent, computers have accomplished some amazing feats, like beating humans at over a dozen Atari games, and defeating the Go world champion.


Learning how to build these agents requires a bit of a change in thinking for anyone used to working in a supervised learning setting though. Gone is the ability to simply get the algorithm to pair certain stimuli with certain responses. Instead RL algorithms must enable the agent to learn the correct pairings itself through the use of observations, rewards, and actions. Since there is no longer a “True” correct action for an agent to take in any given circumstance that we can just tell it, things get a little tricky. In this post and those to follow, I will be walking through the creation and training of reinforcement learning agents. The agent and task will begin simple, so that the concepts are clear, and then work up to more complex task and environments.

## Two-Armed Bandit

The simplest reinforcement learning problem is the n-armed bandit. Essentially, there are n-many slot machines, each with a different fixed payout probability. The goal is to discover the machine with the best payout, and maximize the returned reward by always choosing it. We are going to make it even simpler, by only having two possible slot machines to choose between. In fact, this problem is so simple that it is more of a precursor to real RL problems than one itself. Let me explain. Typical aspects of a task that make it an RL problem are the following:
- Different actions yield different rewards. For example, when looking for treasure in a maze, going left may lead to the treasure, whereas going right may lead to a pit of snakes.
- Rewards are delayed over time. This just means that even if going left in the above example is the right things to do, we may not know it till later in the maze.
- Reward for an action is conditional on the state of the environment. Continuing the maze example, going left may be ideal at a certain fork in the path, but not at others.

The n-armed bandit is a nice starting place because we don’t have to worry about aspects #2 and #3. All we need to focus on is learning which rewards we get for each of the possible actions, and ensuring we chose the optimal ones.

In the context of RL lingo, this is called learning a policy. We are going to be using a method called **policy gradients**, where our simple neural network learns a policy for picking actions by **adjusting it’s weights** through **gradient descent** using feedback from the environment. There is another approach to reinforcement learning where agents learn value functions. In those approaches, instead of learning the optimal action in a given state, the agent learns to predict how good a given state or action will be for the agent to be in. Both approaches allow agents to learn good behavior, but the policy gradient approach is a little more direct.

### Policy Gradient
The simplest way to think of a Policy gradient network is one which produces explicit outputs. In the case of our bandit, we don’t need to condition these outputs on any state. As such, our network will consist of just a set of weights, with each corresponding to each of the possible arms to pull in the bandit, and will represent how good our agent thinks it is to pull each arm. If we initialize these weights to 1, then our agent will be somewhat optimistic about each arm’s potential reward.

To update our network, we will simply try an arm with an e-greedy policy. This means that most of the time our agent will choose the action that corresponds to the largest expected value, but occasionally, with $e$ probability, it will choose randomly. In this way, the agent can try out each of the different arms to continue to learn more about them. Once our agent has taken an action, it then receives a reward of either $1 or -1$. With this reward, we can then make an update to our network using the policy loss equation:

$$Loss = -log(\Pi)*A$$

$A$ is advantage, and is an essential aspect of all reinforcement learning algorithms. Intuitively it corresponds to how much better an action was than some baseline. In future algorithms, we will develop more complex baselines to compare our rewards to, but for now we will assume that the baseline is 0, and it can be thought of as simply the reward we received for each action.

$\Pi$ is the policy. In this case, it corresponds to the chosen action’s weight.

Intuitively, this loss function allows us to **increase the weight** for actions that yielded a **positive reward**, and **decrease** them for actions that yielded a **negative reward**. In this way the agent will be more or less likely to pick that action in the future. By taking actions, getting rewards, and updating our network in this circular manner, we will quickly converge to an agent that can solve our bandit problem! Don’t take my word for it though. Try it out yourself.

In [93]:
import math

math.log(1), math.log(0.001), math.log(2), math.log(3), math.log(10), math.log(200)

(0.0,
 -6.907755278982137,
 0.6931471805599453,
 1.0986122886681098,
 2.302585092994046,
 5.298317366548036)

## The Multi-armed bandit
This tutorial contains a simple example of how to build a policy-gradient based agent that can solve the multi-armed bandit problem.

In [1]:
import tensorflow as tf
import numpy as np

/home/mageswarand/anaconda3/envs/tensorflow1.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### The Bandits
Here we define our bandits. For this example we are using a four-armed bandit. The pullBandit function generates a random number from a normal distribution with a mean of 0. The lower the bandit number, the more likely a positive reward will be returned. We want our agent to learn to always choose the bandit that will give that positive reward.

In [15]:

#List out our bandits. Currently bandit 4 (index#3) is set to most often provide a positive reward.
#List out our bandit arms. 
#Currently arm 4 (index #3) is set to most often provide a positive reward.
bandit_arms = [0.2,0,-0.2,-2] #the lesser the value the more the chances of getting higher value than that

NUM_BANDITS = len(bandit_arms)

def pullBandit(bandit):
    #Get a random number.
    result = np.random.randn(1)
    if result > bandit:
        #return a positive reward.
        return 1
    else:
        #return a negative reward.
        return -1
    

### The Agent   
The code below established our simple neural agent. It consists of a set of values for each of the bandits. Each value is an estimate of the value of the return from choosing the bandit. We use a policy gradient method to update the agent by moving the value for the selected action toward the recieved reward.

In [36]:
tf.reset_default_graph()

#These two lines established the feed-forward part of the network. 
weights = tf.Variable(tf.ones([NUM_BANDITS]))
output = tf.nn.softmax(weights)


#The next six lines establish the training proceedure. We feed the reward and chosen action into the network
#to compute the loss, and use it to update the network.
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)

responsible_output = tf.slice(output,action_holder,[1])

loss = -(tf.log(responsible_output)*reward_holder)

optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)


In [37]:
print(weights)
print(output)

<tf.Variable 'Variable:0' shape=(4,) dtype=float32_ref>
Tensor("Reshape_1:0", shape=(4,), dtype=float32)


### Training the Agent
We will train our agent by taking actions in our environment, and recieving rewards. Using the rewards and actions, we can know how to properly update our network in order to more often choose actions that will yield the highest rewards over time.

https://en.wikipedia.org/wiki/Boltzmann_distribution

In [39]:
tmp = [0.1,0.2,0.4,0.3]
np.random.choice(tmp,p=tmp), np.argmax(tmp == np.random.choice(tmp,p=tmp))

(0.40000000000000002, 1)

In [40]:
total_episodes = 1000 #Set total number of episodes to train agent on.
total_reward = np.zeros(NUM_BANDITS) #Set scoreboard for bandit arms to 0.

init = tf.global_variables_initializer()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        #Choose action according to Boltzmann distribution.
        actions = sess.run(output)
        a = np.random.choice(actions, p=actions)
        action = np.argmax(actions == a) 
        #with "a" the randomness is assured though the probability of getting same index is high
        #action = np.argmax(actions) #try this if you want

        reward = pullBandit(bandit_arms[action]) #Get our reward from picking one of the bandit arms.
        
        #Update the network.
        _,resp,ww = sess.run([update,responsible_output,weights], 
                             feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #Update our running tally of scores.
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " + str(NUM_BANDITS) + " arms of the bandit: " + str(total_reward))
        i+=1
        
        
print("\nThe agent thinks arm " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 arms of the bandit: [ 1.  0.  0.  0.]
Running reward for the 4 arms of the bandit: [  3.  -2.  10.  14.]
Running reward for the 4 arms of the bandit: [  4.  -3.  14.  26.]
Running reward for the 4 arms of the bandit: [ -2.   1.  14.  34.]
Running reward for the 4 arms of the bandit: [ -8.  -8.  19.  42.]
Running reward for the 4 arms of the bandit: [-16. -10.  18.  51.]
Running reward for the 4 arms of the bandit: [-20.  -5.  20.  64.]
Running reward for the 4 arms of the bandit: [-22. -17.  22.  72.]
Running reward for the 4 arms of the bandit: [-23. -12.  28.  86.]
Running reward for the 4 arms of the bandit: [ -22.  -13.   32.  100.]
Running reward for the 4 arms of the bandit: [ -24.  -12.   35.  120.]
Running reward for the 4 arms of the bandit: [ -25.   -9.   36.  139.]
Running reward for the 4 arms of the bandit: [ -31.   -7.   46.  153.]
Running reward for the 4 arms of the bandit: [ -33.   -3.   48.  169.]
Running reward for the 4 arms of the bandit: [

### Lets RIP the above code

In [41]:
with tf.Session() as sess:
    dummy_weigths = [[2., 5., 4., 3.]]
    softmaxed = tf.nn.softmax(dummy_weigths)
    scaled = tf.log(softmaxed)
    print("softmaxed: ", softmaxed.eval())
    print("scaled: ", scaled.eval())
    
    #Uses the scaled logits index as the input for sampling space
    dummy_samples = tf.multinomial(scaled, 5)
    print(dummy_samples.eval()) #always selects the maximum weight index 
    
    dummy_samples = tf.multinomial(tf.log(tf.nn.softmax(dummy_weigths)), 1).eval()
    print("dummy_samples: ", dummy_samples)
    print("chosen_action index: ", dummy_samples[0][0])
    
    dummy_possible_actions = tf.convert_to_tensor([10, 11, 12, 13])  # indices for possible actions
    print("chosen_action: ", dummy_possible_actions[dummy_samples[0][0]].eval())

softmaxed:  [[ 0.0320586   0.64391422  0.23688281  0.08714432]]
scaled:  [[-3.44018984 -0.44018975 -1.44018972 -2.44018984]]
[[1 2 1 1 3]]
dummy_samples:  [[2]]
chosen_action index:  2
chosen_action:  12
